# RESCAL vs BayesianRESCAL on KINSHIP dataset

## 1. Bayesian RESCAL model

Here's the generative process of triple (entity, relation, entity):

For each entity $i$, a latent representation of the entity $e_i \in \mathbb{R}^{D}$ is drawn from the zero mean multivariate-normal distribution with diagonal covariance matrix:

$$e_i \sim N(0, \sigma_e^2 I_D)$$

For each relation $k$, relation matrix $R_k \in \mathbb{R}^{D\times D}$ is drawn from the matrix normal distribution with diagonal covariance matrices:

$$R_k \sim MN(0, \sigma_r^2 I_D, I_D)$$

For each triple $(i, k, j)$, the observed value of the triple $x_{i,k,j}$ is drawn from the normal distribution with bilinear mean and variance:

$$x_{ikj} \sim N(e_i^\top R_k e_j, \sigma_x^2)$$


In [1]:
import numpy as np
import logging
from scipy.io.matlab import loadmat
from scipy.sparse import csr_matrix
import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score

import rescal
from brescal import BayesianRescal

%matplotlib inline

#logger = logging.getLogger()
#logger.setLevel(logging.DEBUG)

In [2]:
mat = loadmat('../data/alyawarradata.mat')
T = np.array(mat['Rs'], np.float32)

max_iter = 20

n_dim = 5
T = np.swapaxes(T, 1, 2)
T = np.swapaxes(T, 0, 1)  # [relation, entity, entity]
n_relation, n_entity, _ = T.shape
print(T.shape)

(26, 104, 104)


## 2. Splitting the kinship dataset into train/test sets

`trainT` only contains a partial observation of the original tensor `T`

In [3]:
import itertools
trainT = np.zeros_like(T)
p = 0.5
for k in range(n_relation):
    for i,j in itertools.product(range(n_entity),repeat=2):
        if T[k, i, j] and np.random.binomial(1, p):
            trainT[k, i, j] = T[k, i, j]

## 3. Training BayesianRESCAL

### 3.1. Grid search on variance parameters

Compare performances of different configuration of variances

#### <font color='red'>RESULT(GRID SEARCH)</font>: BayesianRESCAL does not work well when variance of x is equal or larger than 1

In [4]:
var_list = [0.001, 0.01, 0.1, 1., 10., 100.]
best_roc = 0
for (var_x, var_e, var_r) in itertools.product(var_list,repeat=3):
    model = BayesianRescal(n_dim, var_e=var_e, var_x=var_x, var_r=var_r)
    model.fit(trainT, max_iter=max_iter)
    _T = model._reconstruct()
    score = roc_auc_score(T.flatten(), _T.flatten())
    print('var_x:{0:3.3f}, var_e:{1:3.3f}, var_r:{2:3.3f}, AUC-ROC:{3:.3f}'.format(var_x, var_e, var_r, score))
    if score > best_roc:
        best_vars = (var_x, var_e, var_r)
        best_model = model
        best_roc = score
        
var_x, var_e, var_r = best_vars
print(best_vars)

var_x:0.001, var_e:0.001, var_r:0.001, AUC-ROC:0.930
var_x:0.001, var_e:0.001, var_r:0.010, AUC-ROC:0.930
var_x:0.001, var_e:0.001, var_r:0.100, AUC-ROC:0.930
var_x:0.001, var_e:0.001, var_r:1.000, AUC-ROC:0.922
var_x:0.001, var_e:0.001, var_r:10.000, AUC-ROC:0.944
var_x:0.001, var_e:0.001, var_r:100.000, AUC-ROC:0.943
var_x:0.001, var_e:0.010, var_r:0.001, AUC-ROC:0.943
var_x:0.001, var_e:0.010, var_r:0.010, AUC-ROC:0.944
var_x:0.001, var_e:0.010, var_r:0.100, AUC-ROC:0.931
var_x:0.001, var_e:0.010, var_r:1.000, AUC-ROC:0.931
var_x:0.001, var_e:0.010, var_r:10.000, AUC-ROC:0.930
var_x:0.001, var_e:0.010, var_r:100.000, AUC-ROC:0.944
var_x:0.001, var_e:0.100, var_r:0.001, AUC-ROC:0.944
var_x:0.001, var_e:0.100, var_r:0.010, AUC-ROC:0.944
var_x:0.001, var_e:0.100, var_r:0.100, AUC-ROC:0.944
var_x:0.001, var_e:0.100, var_r:1.000, AUC-ROC:0.944
var_x:0.001, var_e:0.100, var_r:10.000, AUC-ROC:0.944
var_x:0.001, var_e:0.100, var_r:100.000, AUC-ROC:0.931
var_x:0.001, var_e:1.000, var_r:0.001

### 3.2. Training RESCAL

In [5]:
X = list()
for k in range(n_relation):
    X.append(csr_matrix(trainT[k]))
    
A, R, f, itr, exectimes = rescal.rescal_als(X, n_dim)

### 3.3. Compare both models by ROC-AUC

#### <font color='red'>RESULT</font>: BayesianRESCAL is comparable with RESCAL

#### ROC-AUC score of BRESCAL

In [6]:
_T = best_model._reconstruct()
print(roc_auc_score(T.flatten(), _T.flatten()))

0.944133926181


#### ROC-AUC score of RESCAL

In [7]:
_X = np.zeros_like(T)
for k in range(T.shape[0]):
    _X[k] = np.dot(np.dot(A, R[k]), A.T)
print(roc_auc_score(T.flatten(), _X.flatten()))

0.943559626861


## 4. Initialize Bayesian_RESCAL with RESCAL

Let's see that Bayesian_RESCAL performs worse than RESCAL because it can't escape from local optimum.

In [8]:
A, R, f, itr, exectimes = rescal.rescal_als(X, n_dim)
model = BayesianRescal(n_dim, var_e=var_e, var_x=var_x, var_r=var_r)
model.n_relations = n_relation
model.n_entities = n_entity
model.E = A
model.R = np.zeros([n_relation, n_dim, n_dim])
for k in range(n_relation):
    model.R[k] = R[k]
model._gibbs(trainT, max_iter=5)

#### ROC-AUC score of BRESCAL initialized by RESCAL

In [9]:
_T = model._reconstruct()
print(roc_auc_score(T.flatten(), _T.flatten()))

0.943522089717


## 5. Place difference variances on observed / unobserved data points

I adopt idea from implicit feedback approach from the following paper in which they place a different variance on observed/unobserved variables (variance of observed variable is smaller than those of unobserved):

*Collaborative Filtering for Implicit Feedback Datasets, Yifan Hu, et al. ICDM2008*

#### <font color='red'>RESULT</font>: BayesianRESCAL with controlled variance (CV_BRESCAL) performs slightly better than BRESCAL.

In [36]:
observed_var = 0.001
unobserved_var = 1.
for i in range(20):
    model = BayesianRescal(n_dim, var_e=var_e, var_x=var_x, var_r=var_r, controlled_var=True, 
                           obs_var=observed_var, unobs_var=unobserved_var)
    model.fit(trainT, max_iter=max_iter)
    _T = model._reconstruct()
    print(roc_auc_score(T.flatten(), _T.flatten()))

0.96603468367
0.954228095078
0.92305263092
0.965225282285
0.962037325525
0.956771656304
0.953578395038
0.959729215416
0.933131698528
0.941166348379
0.966307331788
0.967347638378
0.953822041785
0.955325682658
0.956017684061
0.934671161096
0.955844075267
0.961994553618
0.964478976009
0.966871021753


In [32]:
observed_var = 0.01
unobserved_var = 10.
for i in range(20):
    model = BayesianRescal(n_dim, var_e=var_e, var_x=var_x, var_r=var_r, controlled_var=True, 
                           obs_var=observed_var, unobs_var=unobserved_var)
    model.fit(trainT, max_iter=max_iter)
    _T = model._reconstruct()
    print(roc_auc_score(T.flatten(), _T.flatten()))

0.946752745901
0.951022258144
0.949277739471
0.940403984303
0.943663270871
0.949165964603
0.954419131324
0.947390551757
0.95124657367
0.947585875852
0.950249743091
0.948909541357
0.951616264297
0.943449345877
0.921866171787
0.948273586321
0.910919494767
0.95790799183
0.949043871343
0.951578882231


In [34]:
observed_var = 0.001
unobserved_var = 10.
for i in range(20):
    model = BayesianRescal(n_dim, var_e=var_e, var_x=var_x, var_r=var_r, controlled_var=True, 
                           obs_var=observed_var, unobs_var=unobserved_var)
    model.fit(trainT, max_iter=max_iter)
    _T = model._reconstruct()
    print(roc_auc_score(T.flatten(), _T.flatten()))

0.909336196341
0.928269713257
0.90857881325
0.868543133987
0.914083476551
0.934207291839
0.892281324341
0.918217649862
0.926920518916
0.888205897972
0.880706106016
0.885897443608
0.935934838012
0.891455154027
0.92816627693
0.919767655984
0.934340133081
0.902647562343
0.906719971984
0.906657896616


In [35]:
observed_var = 0.001
unobserved_var = 100.
for i in range(20):
    model = BayesianRescal(n_dim, var_e=var_e, var_x=var_x, var_r=var_r, controlled_var=True, 
                           obs_var=observed_var, unobs_var=unobserved_var)
    model.fit(trainT, max_iter=max_iter)
    _T = model._reconstruct()
    print(roc_auc_score(T.flatten(), _T.flatten()))

0.682337007569
0.639304674764
0.642063749628
0.596862736435
0.592735167711
0.595027997806
0.688270381752
0.803068396661
0.699368893833
0.82732955638
0.675697237709
0.79420091759
0.605186305543
0.673282545858
0.614933612245
0.720779307686
0.686679387371
0.647647091524
0.682062844833
0.575856356785


## A. Parameter optimization for RESCAL

RESCAL also has some parameters to be tuned.

#### <font color='red'>RESULT</font>: RESCAL is highly robust w.r.t the parameters...

In [38]:
var_list = [0.001, 0.01, 0.1, 1., 10., 100.]
best_roc = 0
for (var_x, var_e, var_r) in itertools.product(var_list,repeat=3):
    A, R, f, itr, exectimes = rescal.rescal_als(X, n_dim, lambda_A=var_x, lambda_R=var_e, lambda_V=var_r)
    _X = np.zeros_like(T)
    for k in range(T.shape[0]):
        _X[k] = np.dot(np.dot(A, R[k]), A.T)
    score = roc_auc_score(T.flatten(), _X.flatten())
    print('var_x:{0:3.3f}, var_e:{1:3.3f}, var_r:{2:3.3f}, AUC-ROC:{3:.3f}'.format(var_x, var_e, var_r, score))
    if score > best_roc:
        best_vars = (var_x, var_e, var_r)
        best_model = model
        best_roc = score
        
lambda_a, lambda_r, lambda_v = best_vars
print(best_vars, best_roc)

var_x:0.001, var_e:0.001, var_r:0.001, AUC-ROC:0.944
var_x:0.001, var_e:0.001, var_r:0.010, AUC-ROC:0.944
var_x:0.001, var_e:0.001, var_r:0.100, AUC-ROC:0.944
var_x:0.001, var_e:0.001, var_r:1.000, AUC-ROC:0.944
var_x:0.001, var_e:0.001, var_r:10.000, AUC-ROC:0.944
var_x:0.001, var_e:0.001, var_r:100.000, AUC-ROC:0.944
var_x:0.001, var_e:0.010, var_r:0.001, AUC-ROC:0.944
var_x:0.001, var_e:0.010, var_r:0.010, AUC-ROC:0.944
var_x:0.001, var_e:0.010, var_r:0.100, AUC-ROC:0.944
var_x:0.001, var_e:0.010, var_r:1.000, AUC-ROC:0.944
var_x:0.001, var_e:0.010, var_r:10.000, AUC-ROC:0.944
var_x:0.001, var_e:0.010, var_r:100.000, AUC-ROC:0.944
var_x:0.001, var_e:0.100, var_r:0.001, AUC-ROC:0.944
var_x:0.001, var_e:0.100, var_r:0.010, AUC-ROC:0.944
var_x:0.001, var_e:0.100, var_r:0.100, AUC-ROC:0.944
var_x:0.001, var_e:0.100, var_r:1.000, AUC-ROC:0.944
var_x:0.001, var_e:0.100, var_r:10.000, AUC-ROC:0.944
var_x:0.001, var_e:0.100, var_r:100.000, AUC-ROC:0.944
var_x:0.001, var_e:1.000, var_r:0.001